In [1]:
!bash -c 'source activate tl-detect && conda install -qy -c conda-forge keras'

Solving environment: ...working... done


==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.5.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda/envs/tl-detect

  added / updated specs: 
    - keras


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    h5py-2.7.1                 |           py35_2         3.9 MB  conda-forge
    keras-2.1.5                |           py35_0         498 KB  conda-forge
    yaml-0.1.7                 |                0         302 KB  conda-forge
    pygpu-0.7.5                |           py35_0         1.4 MB  conda-forge
    mako-1.0.7                 |           py35_0         115 KB  conda-forge
    scipy-1.0.1                |py35_blas_openblas_200        39.4 MB  conda-forge
    markupsafe-1.0             |           py35_0          32

In [29]:
!cat ~/.keras/keras.json

{
    "backend": "tensorflow",
    "image_data_format": "channels_last",
    "epsilon": 1e-07,
    "floatx": "float32"
}

In [24]:
!cp -R ../imgs/training/bosch/Green ../imgs/training/ && cp -R ../imgs/training/bosch/Red ../imgs/training/ && cp -R ../imgs/training/bosch/Yellow ../imgs/training/  && mv ../imgs/training/bosch ../imgs/bosch_tl 

cp: cannot stat '../imgs/training/bosch/Green': No such file or directory


In [ ]:
! cd ../imgs/training && find . -size 0 -exec rm {} \;

In [35]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import keras

# dimensions of our images.
img_width, img_height = 75, 40


In [36]:
UNKNOWN = 4
GREEN = 2
YELLOW = 1
RED = 0

In [37]:
import cv2
import os
import numpy as np
import logging


In [38]:
def read_img(img_path, dim, colspace = 'BGR'):
    if colspace != 'GRAY':
        img = cv2.imread(img_path)
    if img is None:
        raise ValueError('Couldn\'t read img: ' + img_path)
    if colspace == 'HSV':
        logging
        img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        
    return cv2.resize(img, dim, interpolation=cv2.INTER_CUBIC)

def read_color(path, color, dim, colspace):
    x = np.stack([read_img(path + f, dim, colspace) for f in os.listdir(path)], axis=0)
    y = np.full(x.shape[0], color)
    return (x, y)

def read_tl_dir(path, dim, colspace):
    x = []
    y = []
    for directory, color in zip(['Red/', 'Yellow/', 'Green/'], [RED, YELLOW, GREEN]):
        x_img, y_img = read_color(os.path.join(path, directory), color, dim, colspace)
        x.append(x_img)
        y.append(y_img)
    
    x = np.concatenate(x)
    y = np.concatenate(y)
    x = x.astype('float32')
    x /= 255
    NUM_CLASSES = 3
    y = keras.utils.to_categorical(y, NUM_CLASSES)
    return x, y
    

def read_train_test_tl(train_path, test_path, dim, colspace):
    x_train, y_train = read_tl_dir(train_path, dim, colspace)
    x_test, y_test = read_tl_dir(train_path, dim, colspace)

    return x_train, y_train, x_test, y_test


train_data_dir = '../imgs/training/'
validation_data_dir = '../imgs/validation/'

x_train, y_train, x_validation, y_validation = read_train_test_tl(train_data_dir, validation_data_dir, dim=(img_height, img_width),
                                                                  colspace='HSV')


In [39]:
x_train.shape

(7246, 75, 40, 3)

In [17]:
batch_size = 1000
epochs = 25

In [40]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_width, img_height)
    x_validation = x_validation.reshape(x_validation.shape[0], 3, img_width, img_height)
    input_shape = (3, img_width, img_heigth)
else:
    x_train = x_train.reshape(x_train.shape[0], img_width, img_height, 3)
    x_validation = x_validation.reshape(x_validation.shape[0], img_width, img_height, 3)
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))
score = model.evaluate(x_validation, y_validation, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights('first_try.h5')

Train on 7246 samples, validate on 7246 samples
Epoch 1/25
7246/7246 [==============================] - 5s 684us/step - loss: 1.1979 - acc: 0.4510 - val_loss: 0.7744 - val_acc: 0.7755
Epoch 2/25
7246/7246 [==============================] - 4s 616us/step - loss: 0.7130 - acc: 0.7219 - val_loss: 0.6096 - val_acc: 0.7804
Epoch 3/25
7246/7246 [==============================] - 4s 616us/step - loss: 0.6191 - acc: 0.7773 - val_loss: 0.5728 - val_acc: 0.7875
Epoch 4/25
7246/7246 [==============================] - 4s 617us/step - loss: 0.5993 - acc: 0.7811 - val_loss: 0.5893 - val_acc: 0.7833
Epoch 5/25
7246/7246 [==============================] - 4s 616us/step - loss: 0.6021 - acc: 0.7815 - val_loss: 0.5696 - val_acc: 0.7872
Epoch 6/25
7246/7246 [==============================] - 4s 616us/step - loss: 0.5863 - acc: 0.7876 - val_loss: 0.5591 - val_acc: 0.7890
Epoch 7/25
7246/7246 [==============================] - 4s 617us/step - loss: 0.5933 - acc: 0.7861 - val_loss: 0.5569 - val_acc: 0.7893
